In [2]:
# [1]
import torch
import json
import os
import logging
import google.generativeai as genai

from sentence_transformers import SentenceTransformer, util

# --- Configuración del Logging ---
log_file = 'rag.log'
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(log_file),
        logging.StreamHandler()
    ]
)

logging.info("Inicio del notebook 'rag.ipynb'.")

# --- Constantes y Rutas ---
# Definimos las rutas a los artefactos que creamos en el notebook anterior.
DATA_DIR = '../../datos_limpios/'
EMBEDDINGS_PATH = os.path.join(DATA_DIR, 'embeddings.pt')
JSON_PATH = os.path.join(DATA_DIR, 'sitios_cali_maestro.json')
MODEL_NAME = 'paraphrase-multilingual-mpnet-base-v2'

# Variable para controlar el dispositivo (GPU si está disponible, si no CPU)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
logging.info(f"Dispositivo seleccionado para los cálculos: {DEVICE}")

/home/nicolas/Escritorio/Inteligencia_Artificial/Inteligencia Artificial/proyecto/Proyecto_IA_Turistica/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-10-07 00:32:56,927 - INFO - Inicio del notebook 'rag.ipynb'.
2025-10-07 00:32:56,929 - INFO - Dispositivo seleccionado para los cálculos: cpu


In [ ]:
# [2]
# --- Carga de la Base de Conocimiento ---

try:
    # 1. Cargar el tensor de embeddings
    # .to(DEVICE) mueve el tensor a la GPU si está disponible, acelerando los cálculos.
    corpus_embeddings = torch.load(EMBEDDINGS_PATH, map_location=DEVICE)
    logging.info(f"Tensor de embeddings cargado desde '{EMBEDDINGS_PATH}' con forma: {corpus_embeddings.shape}")

    # 2. Cargar la lista de sitios desde el archivo JSON
    with open(JSON_PATH, 'r', encoding='utf-8') as f:
        corpus_sitios = json.load(f)
    logging.info(f"Se cargaron {len(corpus_sitios)} registros de sitios desde '{JSON_PATH}'.")

    print("Base de conocimiento cargada exitosamente.")
    
except FileNotFoundError as e:
    logging.error(f"No se pudo encontrar un archivo necesario: {e}")
    print(f"Error: No se encontró el archivo '{e.filename}'. Asegúrate de haber ejecutado el notebook 'embedding.ipynb' primero.")
    corpus_embeddings, corpus_sitios = None, None

2025-10-07 00:11:00,788 - INFO - Tensor de embeddings cargado desde '../../datos_limpios/embeddings.pt' con forma: torch.Size([2470, 768])


2025-10-07 00:11:00,889 - INFO - Se cargaron 2470 registros de sitios desde '../../datos_limpios/sitios_cali_maestro.json'.


Base de conocimiento cargada exitosamente.


In [ ]:
# [3]
# --- Carga del Modelo Codificador (Encoder) ---
# Debe ser exactamente el mismo modelo usado para crear los embeddings del corpus.

try:
    logging.info(f"Cargando el modelo codificador '{MODEL_NAME}'...")
    model = SentenceTransformer(MODEL_NAME, device=DEVICE)
    logging.info("Modelo codificador cargado exitosamente.")
    print("Modelo codificador cargado exitosamente.")
except Exception as e:
    logging.critical(f"No se pudo cargar el modelo de SentenceTransformer. Error: {e}")
    model = None

2025-10-07 00:11:00,903 - INFO - Cargando el modelo codificador 'paraphrase-multilingual-mpnet-base-v2'...
2025-10-07 00:11:00,908 - INFO - Load pretrained SentenceTransformer: paraphrase-multilingual-mpnet-base-v2
2025-10-07 00:11:00,908 - INFO - Load pretrained SentenceTransformer: paraphrase-multilingual-mpnet-base-v2
2025-10-07 00:11:05,373 - INFO - Modelo codificador cargado exitosamente.
2025-10-07 00:11:05,373 - INFO - Modelo codificador cargado exitosamente.


Modelo codificador cargado exitosamente.


In [ ]:
# [4]
# --- Función del Retriever (Búsqueda de Similitud) ---

def buscar_sitios_relevantes(query: str, top_k: int = 5):
    """
    Toma una consulta de usuario, la convierte en un embedding y encuentra los 'top_k' sitios más similares.
    """
    if corpus_embeddings is None or model is None:
        logging.error("La base de conocimiento o el modelo no están cargados. No se puede realizar la búsqueda.")
        return []

    logging.info(f"Recibida nueva consulta: '{query}'")
    
    # 1. Codificar la consulta del usuario en un vector (embedding)
    query_embedding = model.encode(query, convert_to_tensor=True, device=DEVICE)
    
    # 2. Calcular la similitud del coseno entre la consulta y todos los embeddings del corpus
    # util.cos_sim es una operación de tensor altamente optimizada.
    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    
    # 3. Encontrar los 'top_k' resultados con las puntuaciones más altas
    # torch.topk nos devuelve tanto las puntuaciones como los índices de los mejores resultados.
    top_results = torch.topk(cos_scores, k=min(top_k, len(corpus_sitios)))
    
    logging.info(f"Búsqueda completada. Se encontraron {len(top_results[0])} resultados relevantes.")
    
    # 4. Formatear y devolver los resultados
    resultados = []
    for score, idx in zip(top_results[0], top_results[1]):
        # idx es la posición del sitio en nuestra lista original 'corpus_sitios'
        sitio_encontrado = corpus_sitios[idx]
        resultados.append({
            "score": score.item(), # Convertimos el tensor de la puntuación a un número simple
            "nombre": sitio_encontrado.get("nombre"),
            "direccion": sitio_encontrado.get("direccion"),
            "comentarios": sitio_encontrado.get("comentarios")
        })
        
    return resultados

In [ ]:
# [5]
# --- Simulación de Consultas de Usuario ---

# Consulta 1: Específica y con sentimiento
consulta_1 = "Quiero ir a un lugar romántico para una cita, que sea elegante y con comida italiana."
resultados_1 = buscar_sitios_relevantes(consulta_1)
print(f"\n--- Resultados para la consulta: '{consulta_1}' ---")
for res in resultados_1:
    print(f"  - Puntuación: {res['score']:.4f} | Nombre: {res['nombre']}")

# Consulta 2: Basada en actividades
consulta_2 = "Un sitio para caminar y ver arte o historia de la ciudad"
resultados_2 = buscar_sitios_relevantes(consulta_2)
print(f"\n--- Resultados para la consulta: '{consulta_2}' ---")
for res in resultados_2:
    print(f"  - Puntuación: {res['score']:.4f} | Nombre: {res['nombre']}")

# Consulta 3: Gastronómica y popular
consulta_3 = "dónde puedo comer la mejor comida típica del valle, como empanadas o lulada"
resultados_3 = buscar_sitios_relevantes(consulta_3)
print(f"\n--- Resultados para la consulta: '{consulta_3}' ---")
for res in resultados_3:
    print(f"  - Puntuación: {res['score']:.4f} | Nombre: {res['nombre']}")

2025-10-07 00:11:05,410 - INFO - Recibida nueva consulta: 'Quiero ir a un lugar romántico para una cita, que sea elegante y con comida italiana.'
Batches: 100%|██████████| 1/1 [00:00<00:00,  5.11it/s]
2025-10-07 00:11:05,622 - INFO - Búsqueda completada. Se encontraron 5 resultados relevantes.
Batches: 100%|██████████| 1/1 [00:00<00:00,  5.11it/s]
2025-10-07 00:11:05,622 - INFO - Búsqueda completada. Se encontraron 5 resultados relevantes.
2025-10-07 00:11:05,627 - INFO - Recibida nueva consulta: 'Un sitio para caminar y ver arte o historia de la ciudad'
2025-10-07 00:11:05,627 - INFO - Recibida nueva consulta: 'Un sitio para caminar y ver arte o historia de la ciudad'



--- Resultados para la consulta: 'Quiero ir a un lugar romántico para una cita, que sea elegante y con comida italiana.' ---
  - Puntuación: 0.7055 | Nombre: storia damore granada
  - Puntuación: 0.6685 | Nombre: afuego mgico
  - Puntuación: 0.6475 | Nombre: piazza by storia damore unicentro cali
  - Puntuación: 0.6418 | Nombre: restaurante casa mar
  - Puntuación: 0.6413 | Nombre: la over


Batches: 100%|██████████| 1/1 [00:00<00:00,  6.10it/s]
2025-10-07 00:11:05,816 - INFO - Búsqueda completada. Se encontraron 5 resultados relevantes.
2025-10-07 00:11:05,818 - INFO - Recibida nueva consulta: 'dónde puedo comer la mejor comida típica del valle, como empanadas o lulada'
2025-10-07 00:11:05,816 - INFO - Búsqueda completada. Se encontraron 5 resultados relevantes.
2025-10-07 00:11:05,818 - INFO - Recibida nueva consulta: 'dónde puedo comer la mejor comida típica del valle, como empanadas o lulada'



--- Resultados para la consulta: 'Un sitio para caminar y ver arte o historia de la ciudad' ---
  - Puntuación: 0.7499 | Nombre: monumento letras de cali
  - Puntuación: 0.7323 | Nombre: museo h tejada
  - Puntuación: 0.7164 | Nombre: lugar a dudas
  - Puntuación: 0.7124 | Nombre: calle de la escopeta
  - Puntuación: 0.7103 | Nombre: parque mirador yo amo a silo


Batches: 100%|██████████| 1/1 [00:00<00:00,  4.88it/s]
2025-10-07 00:11:06,049 - INFO - Búsqueda completada. Se encontraron 5 resultados relevantes.
Batches: 100%|██████████| 1/1 [00:00<00:00,  4.88it/s]
2025-10-07 00:11:06,049 - INFO - Búsqueda completada. Se encontraron 5 resultados relevantes.



--- Resultados para la consulta: 'dónde puedo comer la mejor comida típica del valle, como empanadas o lulada' ---
  - Puntuación: 0.7389 | Nombre: chicharritos antojos de tradicin
  - Puntuación: 0.7043 | Nombre: fritanga las collazos
  - Puntuación: 0.6867 | Nombre: antojitos tipicos del valle restaurante cali norte
  - Puntuación: 0.6732 | Nombre: los antojos de yayis
  - Puntuación: 0.6716 | Nombre: ricuras de la 12


In [ ]:
# [6]
# --- Configuración y DIAGNÓSTICO de la API de Google Gemini ---
from dotenv import load_dotenv

dotenv_path = os.path.join(os.getcwd(), '../../.env/.env')
load_dotenv(dotenv_path=dotenv_path)

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

if GOOGLE_API_KEY:
    try:
        genai.configure(api_key=GOOGLE_API_KEY)
        logging.info("API de Google Gemini configurada exitosamente.")
        print("API de Google Gemini configurada exitosamente.")

        # --- ¡ESTA ES LA PARTE IMPORTANTE PARA DEPURAR! ---
        # Listamos todos los modelos que soportan 'generateContent' para tu clave API.
        print("\n--- Modelos de Gemini Disponibles para tu API Key ---")
        for m in genai.list_models():
            if 'generateContent' in m.supported_generation_methods:
                print(m.name)

    except Exception as e:
        logging.critical(f"No se pudo configurar la API de Gemini. Error: {e}")
        print(f"Error crítico al configurar la API de Gemini. Verifica tu clave API. Error: {e}")
else:
    logging.warning("No se encontró la variable de entorno 'GOOGLE_API_KEY'.")
    print("Advertencia: No se encontró la clave API de Google.")

2025-10-07 00:11:06,085 - INFO - API de Google Gemini configurada exitosamente.
E0000 00:00:1759813866.087855  103006 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
E0000 00:00:1759813866.087855  103006 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


API de Google Gemini configurada exitosamente.

--- Modelos de Gemini Disponibles para tu API Key ---
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-05-20
models/gemini-2.5-flash
models/gemini-2.5-flash-lite-preview-06-17
models/gemini-2.5-pro-preview-05-06
models/gemini-2.5-pro-preview-06-05
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-preview-image-generation
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinking-exp-1219
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/learnlm-2.0-flash-experimental
models/gemma-3-1b-it
models/gemma-3-

In [ ]:
# [7]
# --- Integración con el Generador (Google Gemini Pro - Corregido Definitivamente) ---

def generar_respuesta_con_gemini(query: str, contexto: list):
    """
    Toma la consulta original y el contexto recuperado para generar una respuesta usando Gemini.
    """
    if not GOOGLE_API_KEY:
        return "Error: La clave API de Google no está configurada. No se puede generar una respuesta."
    if not contexto:
        return "Lo siento, no pude encontrar ninguna recomendación relevante para tu consulta en nuestra base de datos."

    # 1. Instanciar el modelo con el alias 'latest' que SÍ está en tu lista de modelos.
    # Corregimos de 'gemini-1.5-pro-latest' a 'gemini-pro-latest'.
    llm_model = genai.GenerativeModel('gemini-pro-latest')
    
    # 2. Construir el prompt detallado para el LLM (esto no cambia)
    prompt = f"""
    Eres "Cali-Guía IA", un asistente de turismo experto, amigable y apasionado por la ciudad de Cali, Colombia.
    Tu objetivo es ayudar a un turista a planificar su visita basándote ÚNICAMENTE en la información que te proporciono.

    La pregunta del turista es: "{query}"

    He encontrado los siguientes lugares que parecen ser muy relevantes. Úsalos como base para tu respuesta:
    """
    for i, sitio in enumerate(contexto):
        prompt += f"\n--- Contexto del Lugar {i+1} ---\n"
        prompt += f"Nombre del Lugar: {sitio['nombre']}\n"
        prompt += f"Ubicación: {sitio['direccion']}\n"
        prompt += f"Extracto de opiniones de visitantes: {sitio['comentarios'][:400]}...\n"
    
    prompt += """
    ---
    Tu Tarea:
    1.  Escribe una respuesta conversacional y atractiva. Empieza con un saludo cálido.
    2.  Recomienda los lugares del contexto que mejor se ajusten a la pregunta del turista.
    3.  Para cada lugar recomendado, menciona su nombre y por qué es una buena opción, basándote en las opiniones o características proporcionadas.
    4.  NO inventes información. Si no tienes detalles sobre algo (ej. horarios, precios exactos), no los menciones.
    5.  Termina con una frase amigable, como "¡Espero que disfrutes tu increíble visita a Cali!".
    """

    # 3. Hacer la llamada a la API y manejar posibles errores (esto no cambia)
    try:
        logging.info(f"Generando respuesta con el modelo '{llm_model.model_name}'...")
        response = llm_model.generate_content(prompt)
        logging.info("Respuesta recibida de la API de Gemini.")
        return response.text
    except Exception as e:
        logging.error(f"Ocurrió un error al llamar a la API de Gemini: {e}")
        return f"Lo siento, hubo un problema al comunicarme con el asistente de IA. Error: {e}"

In [ ]:
# [8]
# --- Ejecución del Flujo RAG Completo con Gemini ---

consulta_final = "Tengo una cena importante con mi pareja, busco un sitio elegante, con buena comida y un ambiente especial"

# Paso 1: Recuperar (Retrieve) - Esto no cambia
contexto_recuperado = buscar_sitios_relevantes(consulta_final, top_k=3)

# Paso 2: Generar (Generate) - Ahora llama a nuestra nueva función
respuesta_final = generar_respuesta_con_gemini(consulta_final, contexto_recuperado)


print("\n\n========================================================")
print("========= EJECUCIÓN DEL FLUJO RAG COMPLETO =========")
print("========================================================")
print(f"\nCONSULTA DE USUARIO:\n'{consulta_final}'\n")
print("CONTEXTO RECUPERADO (lo que ve el LLM):")
for i, sitio in enumerate(contexto_recuperado):
    print(f"  {i+1}. {sitio['nombre']} (Score: {sitio['score']:.3f})")

print("\nRESPUESTA GENERADA POR GEMINI PRO:")
print(respuesta_final)

2025-10-07 00:11:06,920 - INFO - Recibida nueva consulta: 'Tengo una cena importante con mi pareja, busco un sitio elegante, con buena comida y un ambiente especial'
Batches: 100%|██████████| 1/1 [00:00<00:00,  3.10it/s]
2025-10-07 00:11:07,293 - INFO - Búsqueda completada. Se encontraron 3 resultados relevantes.
2025-10-07 00:11:07,296 - INFO - Generando respuesta con el modelo 'models/gemini-pro-latest'...

2025-10-07 00:11:07,293 - INFO - Búsqueda completada. Se encontraron 3 resultados relevantes.
2025-10-07 00:11:07,296 - INFO - Generando respuesta con el modelo 'models/gemini-pro-latest'...
E0000 00:00:1759813867.301317  103006 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
E0000 00:00:1759813867.301317  103006 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
2025-10-07 00:11:30,056 - INFO - Respuesta recibida de la API de Gemini.
2025-10-07 00:11:30,056 - INFO - Respuesta recibida



========= EJECUCIÓN DEL FLUJO RAG COMPLETO =========

CONSULTA DE USUARIO:
'Tengo una cena importante con mi pareja, busco un sitio elegante, con buena comida y un ambiente especial'

CONTEXTO RECUPERADO (lo que ve el LLM):
  1. restaurante casa mar (Score: 0.722)
  2. asados temos (Score: 0.705)
  3. afuego mgico (Score: 0.696)

RESPUESTA GENERADA POR GEMINI PRO:
¡Hola! Soy Cali-Guía IA, ¡qué bueno poder ayudarte a encontrar el lugar perfecto en mi querida Cali para esa cena tan especial! Una velada romántica en la Sucursal del Cielo es un plan maravilloso.

Basándome en lo que buscas, aquí te presento las opciones que mejor se ajustan para crear un momento inolvidable:

1.  **afuego mgico:** Este lugar parece ser exactamente lo que tienes en mente. Según las opiniones de otros visitantes, es **"un lugar hermoso y especial para compartir con tu pareja"**. Es ideal si lo que quieres es sorprenderla con una **"cena espectacular"**. Además, destacan que tiene **"un buen servicio y comi